# **Setup**
 
Reset the Python environment to clear it of any previously loaded variables, functions, or libraries. Then, import the libraries needed to complete the code Professor Melnikov presented in the video.

In [ ]:
%reset -f
from IPython.core.interactiveshell import InteractiveShell as IS
IS.ast_node_interactivity = "all"    # allows multiple outputs from a cell
import numpy as np, pandas as pd, seaborn as sns, matplotlib.pyplot as plt, scipy, re
from sentence_transformers import SentenceTransformer
pd.set_option('max_rows', 5, 'max_columns', 20, 'max_colwidth', 100, 'precision', 2)

<hr style="border-top: 2px solid #606366; background: transparent;">

# **Review**

Review the code Professor Melnikov used to generate a medoid for the set of movie vectors.


## **Load TMDB and SBERT**

Recall that the Movie Database ([TMDB](https://www.themoviedb.org/)) file (`movies.zip`) contains 4803 movies (rows) and 19 features (columns), which can be textual and numeric. It is loaded below into `df` dataframe with title as index labels.

<span style="color:black"> The code below also loads the Sentence Transformer ([SBERT](https://www.sbert.net/) ), which converts any length text to a fixed size numeric vector of 768 dimensions. Unlike in the video, a much smaller pre-trained language model, `'paraphrase-albert-small-v2'`, is loaded here for the sake of speed. It is 50 MB, which is about seven times smaller than the one in the video. Note that in a production environment, the speed needs to be balanced with the embedding quality, which in itself has to be mathematically sound.

In [ ]:
%time SBERT = SentenceTransformer('paraphrase-albert-small-v2')   # load a pre-trained model, 250MB
df0 = pd.read_csv('movies.zip').fillna('')  # load TMDB file
df0.index = df0.title        # set index labels for visual convenience only

## **Filter Movies and Build More Comprehensive Descriptions**

<span style="color:black"> In the video, 1154 `Action` genre movies were used. Here, to speed up the encoding, the cell below further selects movies that are both `Action` and `Family` genre movies, leaving only 62 films for processing.
    
<span style="color:black"> Also, the code below concatenates title, tagline, and overview of each movie into a `Desc` field, which will later be encoded. The hypothesis is that more diverse and descriptive text gives better results. However, the embeddings are unlikely to improve if you concatenate the `overview` attribute 100 times. The JSON parsing seen in the video is left out to shorten the setup.

In [ ]:
dfS = df0[df0.genres.str.contains('Action') & df0.genres.str.contains('Family')]   # sample of movies
df = (dfS.title +'. '+ dfS.tagline +'. '+ dfS.overview).to_frame().rename(columns={0:'Desc'})
df

## **Encode Movies With SBERT Language Model**

The next cell passes a list of all movie descriptions through the embedding model and packages the results as a numeric dataframe with all coefficients packed in 768 columns. Each row is a numeric vector for the given movie with a title in the row index.

In [ ]:
%time mEmb = SBERT.encode(df.Desc) # Encoding textual descriptions with numeric vectors
dfEmb = pd.DataFrame(mEmb, index=df.index)
dfEmb

## **Compute Medoid, a Representative Movie**

<span style="color:black"> Next, `GetMedoidIndex()` is a UDF that takes a matrix of movie embeddings and returns the index of the row with the most representative vector.

In [ ]:
def GetMedoidIndex(vX):
    vMean = np.mean(vX, axis=0)                           # compute centroid
    return np.argmin([sum((x - vMean)**2) for x in vX])   # pick a point closest to centroid

nIX = GetMedoidIndex(dfEmb.values)      # store index of the medoid 
print(f'Medoid movie: {df.index[nIX]}')

## **Compress Dimensions for Plotting**

<span style="color:black">Principal Component Analysis (PCA) is a powerful technique for compressing dimensions under specific constraints. Simply said, PCA is a dimension reduction technique, which you will use here to approximate 768 dimensions with just two dimensions. Here, you use it to rotate the movie vectors such that the first dimension (or first principal component, PC1) captures most of the variability in embedding coefficients of the movie vectors. Then, PC2 captures most of the remaining variability and so on. All but the first to principal components, which are needed for 2D plotting, are ignored.
    
<span style="color:black">While the theory behind PCA can be fairly exciting and involved, its application is straight forward. You create a `PCA()` object and specify the desired number of  PCs to return. Then, the object is fitted on the matrix `dfEmb` and 62 rows are returned with two columns. These columns are called $x$ and $y$, and are coordinates of each movie on a 2D plane.

In [ ]:
from sklearn.decomposition import PCA   # PCA uses SVD to reduce dimensionality of the feature space
# ?PCA                                  # to view help manual
mPC12 = PCA(n_components=2).fit_transform(dfEmb)   # project 768-dim vectors to 2D space for plotting
dfPC12 = pd.DataFrame(mPC12, columns=['x','y'], index=df.index)
dfPC12                                  # contains new (x,y) coordinates and cluster label

## **Plot the Movies and Medoid**

<span style="color:black">Finally, the movies are plotted with their descriptions (using plotly objects which is not covered by this course) and the medoid movie is identified. Note that, as expected, it is somewhat in the middle of the cluster. Whether this movie is representative of all given action and family films is to be determined by an expert.
    
<span style="color:black">  On the left, there are spy/cop/detective movies, while the movies appear more diverse, but many relate to nature/science/art. The medoid, 'Spy Kids 2' seems to be a blend between science and spy/detective genre based on the description used to create its vector. It makes sense to represent the movie cluster at hand. If the given medoid did not seem to be representative, you would need to look into a different embedding model and/or different movie descriptions/attributes.

In [ ]:
import plotly.graph_objects as go       # import graph object from plotly library
print('Action-Familiy movies with a medoid')

x, y = dfPC12.iloc[nIX]
sMovieGenres = df.Desc.apply(lambda s: '<br>'.join(re.findall('.{1,%i}' % 50, s))) # point labels with title+genre
DMarkers = dict(size=2, line=dict(width=1, color='brown'), color='tan')
goMargin = go.layout.Margin(l=0, r=0, b=0, t=0)
goS = go.Scatter(x=dfPC12.x, y=dfPC12.y, mode='markers', marker=DMarkers, text=sMovieGenres, name='movies');
goMedoid = go.Scatter(x=[x], y=[y], name='medoid', marker=dict(size=10, color='orange'), text=sMovieGenres[nIX])
goLayout = go.Layout(hovermode='closest', margin=goMargin, width=1000, 
                   height=500, xaxis={'title':'PC1'}, yaxis={'title':'PC2'});

fig = go.Figure(layout=goLayout)  # prepare a figure with specified layout
fig = fig.add_trace(goMedoid)           # add medoid
fig.add_trace(goS)                # add movie points to canvas

<hr style="border-top: 2px solid #606366; background: transparent;">

# **Optional Practice**

Now you will practice finding a representative movie. 

As you work through these tasks, check your answers by running your code in the *#check solution here* cell, to see if you’ve gotten the correct result. If you get stuck on a task, click the **See solution** drop-down to view the answer.

## Task 1

<span style="color:black"> For the set of action-family movies, use encoding to find the medoid movie title for each of the following: 1) movie titles, 2) movie taglines, and 3) movie overviews. Which medoid appears most reasonable? What differences do you notice for the encoding times?
    
<b>Hint:</b> You only need to encode a title column with the loaded <code>SBERT</code> model and pass results through <code>GetMedoidIndex()</code>. Then use the resulting medoid index to extract the movie title. Then repeat for tagline and overview columns. You can time the encoding with <code>%time</code>.

In [ ]:
# check solution here

<font color=#606366>
    <details><summary><font color=#B31B1B>▶ </font>See <b>solution</b>.</summary>
<pre class="ec">
%time df.index[GetMedoidIndex(SBERT.encode(dfS.title))]
%time df.index[GetMedoidIndex(SBERT.encode(dfS.tagline))]
%time df.index[GetMedoidIndex(SBERT.encode(dfS.overview))]
            </pre>The medoid titles: 'Teenage Mutant Ninja Turtles II: The Secret of the Ooze', 'The Nutcracker: The Untold Story', 'Spy Kids: All the Time in the World'. Each medoid could be reasonable within its corpus, but Spy Kids movie appears closest to Spy Kids 2 we discovered earlier. Notably, with more descriptive text, the medoid appears more representative of the film content. In part, this is because a shorter text, such as a movie title or tagline, is greatly limited in its ability to describe the full story in the video. CPU times: 4 sec, 7 sec, 9 sec. It takes longer (but not proportionally) to encode longer text. While overviews are, perhaps, 10x longer than movie titles, encoding the former takes only about twice as long.
</details> 
</font>

<hr>

## Task 2

Find a medoid title from the overview text of movies with Animation and Drama genres (that's 19 films). Does this movie appear representative for this subsample of movies? Why or why not?

<b>Hint:</b> First, use <code>df0.genres.str.contains</code> filter to re-sample the movies. Then encode the overview column with the loaded SBERT model. Then compute the medoid index and use it to find the medoid movie title.

In [ ]:
# check solution here

<font color=#606366>
    <details><summary><font color=#B31B1B>▶ </font>See <b>solution</b>.</summary>
<pre class="ec">
df1 = df0[df0.genres.str.contains('Animation') & df0.genres.str.contains('Drama')]   # filter to action+history movies only
%time df1.title[GetMedoidIndex(SBERT.encode(df1.overview))]
pd.set_option('max_rows', 20, 'max_colwidth', 1000)
df1[['overview']].sort_values('title')
            </pre>To make a judgment whether 'A Cristmas Carol' description is a reasonable representation for the sample of overviews, you need to "become" an expert in these movie descriptions. Spend some time to read them and group them based on your understanding of the descriptive content.
</details> 
</font>

<hr>